In [1]:
import re
import requests
import math
from lxml import html
from collections import Counter
from string import punctuation as enPunc
from zhon.hanzi import punctuation as zhPunc

import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
from bs4  import BeautifulSoup
from LAC import LAC
from opencc import OpenCC

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/opt/anaconda3/lib/python3.8/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


In [2]:
'''
Question: what's the import data look like?
Currently an Excel.
* Related to how to read the links.
'''
search_word = '振华数据'
google_search_rs = pd.read_excel('search2.xlsx').url.to_list()

## Target Page: Term Frequency Indexing

In [3]:
def getLink(google_search_rs):
    getLink = []
    for idx, link in enumerate(google_search_rs):
        getLink.append((idx+1, link))
    return getLink

In [4]:
def getFinalText(google_search_rs):
    html_text = []
    for idx, link in enumerate(google_search_rs):
        resp = requests.get(link, headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'})
        soup = BeautifulSoup(resp.content, 'html.parser')
        a_tag = soup.find_all('a')
        for a in a_tag:
            a.decompose()   
        html_text.append(soup.get_text()) 
    
    # 繁轉簡
    cc = OpenCC('tw2sp')
    temp = [cc.convert(texts) for texts in html_text]

    # 去除英文
    final_text = [re.sub(r'[a-zA-Z]',"", texts) for texts in temp]
    return final_text

In [5]:
def multi_repl(input_str):
    trim = ['\n', '\r', '\t', '\xa0', ' '] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim_dict = {pat:'' for pat in trim}
    for k in trim_dict:
        input_str = input_str.replace(k, trim_dict[k])  ## 把stop words改為空白
    return input_str

In [6]:
def stopwords():
    # Load stop words
    f = open('/Users/csti-user/Downloads/NER/stopwords-master/baidu_stopwords.txt','r')
    k = f.readlines()
    f.close()
    stopwords = [lines.strip('\n') for lines in k]

    # 建構stopwords
    trim = ['\n', '\r', '\t', '\xa0'] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim = trim + stopwords  # list of all stopwords
    return trim

In [7]:
def ner_combine(ner_pair):
    for idx, (word, ner, cat) in enumerate(ner_pair):
        if (idx + 1) == len(ner_pair) or (idx + 2) == len(ner_pair):
            break
        else:  # 直接將組合詞加入原先分詞好的list (ner_pair)
            n1_ner = ner_pair[idx+1]
            if cat == n1_ner[2]:
                n2_ner = ner_pair[idx+2]
                if ner == 'ORG' and n1_ner[1] == 'PER':
                    new_wd = (word+n1_ner[0],'TITLE+PER',cat)
                    del ner_pair[idx:idx+2]
                    ner_pair.insert(idx,new_wd)
#                     n = idx-1
#                     while n < idx:
#                         if ner_pair[n][1] == 'Full':
#                             break
#                         if ner_pair[n][1] != 'ORG':
#                             n-=1
#                         elif ner_pair[n][1] == 'ORG':
#                             new_wd = (''.join(str(k[0]) for k in ner_pair[n:idx+1]), 'Full', ner_pair[idx][2])
#                             del ner_pair[n:idx+1]
#                             ner_pair.insert(n, new_wd)
#                             break      
                elif ner == 'ORG' and n1_ner[1] == 'TITLE' and n2_ner[1] == 'PER':
                    new_wd = (word+n1_ner[0]+n2_ner[0],'ORG+TITLE+PER',cat)
                    del ner_pair[idx:idx+3]
                    ner_pair.insert(idx,new_wd)

In [8]:
def ner_extract(final_text):
    lac = LAC(mode='lac')  
    lac.load_customization('custom_dict.txt')  # 自行增加字典
    ner_model = lac.run(final_text)

    ner_pair = []
    ner_wd = []
    trim = stopwords()
#     docs = pd.DataFrame(columns=['texts', 'doc'])  # dataframe 每篇文章為一列
    for doc, (word, ner) in enumerate(ner_model):
        temp = []
        for num in range(len(word)):
            if word[num] not in trim: # remove stop words
                word[num] = multi_repl(word[num])
                if word[num] != '' and ner[num] != '' and ner[num] != 'f' and ner[num] != 'nw' and ner[num] != 'm':
                    temp.append(word[num])
                    ner_pair.append((word[num],ner[num],doc+1))
        ner_wd.append(" ".join(str(x) for x in temp))
#         docs = docs.append({'texts': temp, 'doc': doc+1}, ignore_index = True)
    ner_combine(ner_pair)
    return ner_pair, ner_wd

In [9]:
## Term Frequency Indexing
def target_page(ner_pair):
    df_all_pair = pd.DataFrame(ner_pair,columns=['words','ner','cat'])
    # dataframe ORG+PER, ORG+TITLE+PER
    df1 = df_all_pair[df_all_pair['ner'] == 'ORG+PER']
    df2 = df_all_pair[df_all_pair['ner'] == 'ORG+TITLE+PER']
    df_ppl_pair = pd.concat((df1,df2),ignore_index=True)
    cnt = Counter(df_ppl_pair.cat).most_common()
    
    link = pd.DataFrame(getLink(google_search_rs), columns=['doc','link'])
    link['cnt'] = 0
    for i in cnt:
        for idx in link.doc:
            if i[0] == idx:
                link['cnt'][idx-1] = i[1]
    link = link.sort_values(by=['cnt'],ascending=False, ignore_index=True)
    link = link.drop(columns=['cnt'])
    return link

In [10]:
final_text = getFinalText(google_search_rs)
ner_pair, ner_wd = ner_extract(final_text)
target_page(ner_pair)

2021-06-16 17:03:29,382 - WARNING - Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
<ipython-input-9-5561b72ead57>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['cnt'][idx-1] = i[1]


,doc,link
0,42,http://www.bast.net.cn/art/2019/12/25/art_2334...
1,50,https://www.tiaotiaowen.com/articles/bsd8wy
2,25,https://www.rfa.org/mandarin/yataibaodao/junsh...
3,46,https://www.taiwanjustice.net/2020/09/15/%E8%8...
4,24,https://www.fountmedia.io/article/75492
5,3,https://tw.appledaily.com/politics/20200920/RA...
6,29,https://www.rti.org.tw/news/view/id/2079508
7,45,https://www.mingjingnews.com/article/195941-01
8,11,https://www.abc.net.au/chinese/2020-09-14/chin...
9,17,https://newsin.us/%E6%8C%AF%E5%8D%8E%E6%95%B0%...


## Single doc information extraction

In [ ]:
'''
萃取方法：詞頻排序
1. 選擇ORG, PER, ORG+TITLE+PER, ORG+PER
2. 程式產出：ORG+TITLE+PER, ORG+PER, ORG, PER 15個詞頻高者 無排序
3. 文章分詞：ORG+TITLE+PER, ORG+PER, ORG, PER, TITLE 15個詞頻高者 按照文章順序排序
＊詞頻法過於直觀，程式產出處很難針對重要性去排列
################################################################################
＊希望能夠做到將文章中「獨特性高」的詞往前排：職稱層級高、單位敏感、名字特別
＊文章分詞處，
'''

In [223]:
def all_term_freq(ner_pair):
    df_all_pair = pd.DataFrame(ner_pair,columns=['words','ner','cat'])
    df_specific = pd.concat((df_all_pair[df_all_pair['ner'] == 'ORG'],df_all_pair[df_all_pair['ner'] == 'PER'], 
               df_all_pair[df_all_pair['ner'] == 'ORG+TITLE+PER'], df_all_pair[df_all_pair['ner'] == 'ORG+PER'])).reset_index()
    df_specific = df_specific.drop(columns=['index']) # 只包含四種ner
    doc_term_freq = []  # 每篇文章每個詞的頻率
    doc_terms = []  # 每篇文章的詞 (normalized)
    for i in range(len(final_text)):
        tempDF = df_all_pair[df_all_pair['cat']==i+1]
        terms = tempDF['words'].unique()
        doc_terms.append(terms)
        cnt = Counter(tempDF.words)
        doc_term_freq.append(cnt)
    return doc_term_freq, df_specific

In [224]:
def singledoc_extract(pair):
    df = pd.DataFrame(pair, columns=['word','ner','doc'])
    ## dataframe including some ner: org,title, per
    keywords = pd.concat((df[df['ner'] == 'ORG+TITLE+PER'],df[df['ner'] == 'ORG+PER'], df[df['ner'] == 'ORG'], df[df['ner'] == 'PER'])).reset_index()
    keywords = keywords.drop(columns=['index'])
    
    ## 15 most common words dataframe
    cnt = pd.DataFrame(Counter(keywords.word).most_common(15),columns=['words', 'cnt'])
    ## add ner
    cnt['ner'] = [keywords.ner[k] for i in range(len(cnt)) for k in range(len(keywords.word.unique())) if cnt.words[i] == keywords.word.unique()[k]]
#     cnt = cnt.drop(columns=['cnt'])
    return cnt

In [225]:
def auto_extract(pair):
    df = pd.DataFrame(pair, columns=['word','ner','doc'])
    ## dataframe including some ner: org,title, per
    keywords = pd.concat((df[df['ner'] == 'ORG+TITLE+PER'],df[df['ner'] == 'ORG+PER'], df[df['ner'] == 'ORG'], 
                          df[df['ner'] == 'PER'])).reset_index()
    keywords = keywords.drop(columns=['index'])
    
    ## 15 most common words dataframe
    cnt = pd.DataFrame(Counter(keywords.word).most_common(15),columns=['words', 'cnt'])
    ## add ner
    cnt['ner'] = [keywords.ner[k] for i in range(len(cnt)) for k in range(len(keywords.word.unique())) if cnt.words[i] == keywords.word.unique()[k]]
    cnt = cnt.drop(columns=['cnt'])
    return cnt

In [226]:
def manual_input(pair):
    df = pd.DataFrame(pair, columns=['word','ner','doc'])
    ## dataframe including some ner: org, title, per
    keywords = pd.concat((df[df['ner'] == 'ORG+TITLE+PER'],df[df['ner'] == 'ORG+PER'], df[df['ner'] == 'ORG'], 
                          df[df['ner'] == 'PER'], df[df['ner'] == 'TITLE'])).sort_index()
    keywords = keywords.reset_index()
    keywords = keywords.drop(columns =['index'])
    
    cnt = auto_extract(pair)
    cntList = cnt.words.unique().tolist()
    
    idx = [i for i in range(len(keywords)) if keywords.word.iloc[i] in cntList]
    keywords = keywords.drop(idx).reset_index()
    keywords = keywords.drop(columns=['index', 'doc'])
    return keywords

In [227]:
## Automatic Word Segmentation and Name Entity Recognition
url = ['http://www.bast.net.cn/art/2019/12/25/art_23348_448376.html'] # select one url
final_text = getFinalText(url)
ner_pair, ner_wd = ner_extract(final_text)  # [('北京科学技术情报学会', 'ORG', 1),....

In [228]:
## programme
'''
程式產出：保留 人物 組織 職稱，計算詞頻前15高者
判斷式：學會後面+title 就合併
'''
auto_extract(ner_pair) # return 15 keywords created automatically

,words,ner
0,北京科学技术情报学会,ORG
1,北京市科学技术协会,ORG
2,中国科协,ORG
3,吴晨生,ORG
4,北京市科学技术协会学会部副部长李斌,ORG+TITLE+PER
5,北京科学技术情报学会理事长谢威,ORG+TITLE+PER
6,科技部中国科学技术信息研究所所长戴国强,ORG+TITLE+PER
7,科技部中国科技发展战略研究院调查统计中心主任邓大胜,ORG+TITLE+PER
8,中国电信研究院技术总监钱兵,ORG+TITLE+PER
9,中国国防科技信息中心总工真溱,ORG+TITLE+PER


In [230]:
## manual input
'''
人工輸入處的分詞結果呈現方式： 
保留org, per, title, org+title+per, org+per，並刪掉已經是前15名的關鍵詞，不排序
'''
manual_input(ner_pair)

,word,ner
0,中共,ORG
1,中国协和医科大学,ORG
2,中国医学科学院基础医学研究所分子生物学国家重点实验室,ORG
3,主任,TITLE
4,研究员,TITLE
5,全国人大,ORG
6,中国工程院,ORG
7,国际医学科学院,ORG
8,北京市科学技术研究院,ORG
9,副书记,TITLE


In [29]:
def add_dict(word, ner):
    path = '/Users/csti-user/Downloads/NER/custom_dict.txt'
    with open(path,'a') as f:
        f.write('%s/%s\n'%(word,ner)) 

In [34]:
add_dict('資策會', 'ORG')

In [ ]:
'''
萃取方法：把原本搜尋結果的idf記錄下來，在此處針對研究人員點擊的文章進行tf-idf關鍵字萃取（鎖定org+title+per, org+per, org, per）
1. 保留原始idf
2. 挑選特定ner
3. 計算該詞的tfidf
4. 排序
'''

In [130]:
# 计算tf
def tf(word, count):
    return count[word] / sum(count.values())

#计算count_list有多少个文件包含word
def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)

# 计算idf
def idf(word, count_list):
    return math.log(len(count_list) / (n_containing(word, count_list)))  

# 计算tf-idf
def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)

In [189]:
url = 'http://www.bast.net.cn/art/2019/12/25/art_23348_448376.html'  # 41
def getIdx(url):
    df = target_page(ner_pair)
    for i in range(len(df)):
        if df.iloc[i,1] == url:
            return df.iloc[i,0]

In [199]:
def extract(url):
    docIdx = getIdx(url) # 41
    doc_term_freq, df_specific = all_term_freq(ner_pair)
    df_wd = df_specific[df_specific['cat']==docIdx]['words'].unique()
    df_tfidf = pd.DataFrame(columns=['word','tfidf'])
    for i, count in enumerate(doc_term_freq):
        if i == docIdx-1:
            for word in count:
                if word in df_wd:
                    df_tfidf = df_tfidf.append({'word':word,'tfidf': tf(word,count)*idf(word,doc_term_freq)},ignore_index=True)
    df_tfidf = df_tfidf.sort_values(by=['tfidf'], ascending=False)
    return df_tfidf

In [202]:
extract(url)[:20]

,word,tfidf
0,北京科学技术情报学会,0.044125
1,北京市科学技术协会,0.026475
4,中国科协,0.017650
20,中国科学技术信息研究所,0.017650
30,中共中央政治局,0.008825
27,辽宁省科学技术情报研究所,0.008825
26,中国国防科技信息中心,0.008825
25,中国社会科学院,0.008825
24,北京市科学技术情报研究院,0.008825
23,中国电信研究院,0.008825
